# Преамбула

In [ ]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds

tfds.disable_progress_bar()
tf.enable_v2_behavior()

print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.experimental.list_physical_devices()

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns

In [ ]:
import copy

In [ ]:
from sklearn.neighbors import KernelDensity
from sklearn.neighbors import BallTree
from sklearn.neighbors import KDTree

from sklearn.metrics import pairwise_distances_argmin_min

from sklearn.decomposition import PCA

from sklearn.model_selection import GridSearchCV

In [ ]:
from joblib import Parallel, delayed

In [ ]:
def normalize_img(image, label):
    """Нормализация изображений: `uint8` -> `float32`."""
    return tf.cast(image, tf.float32) / 255.0, label

In [ ]:
def crop_pixels(x):
    """Обрезание значений пикселей нормированного изображения."""
    return min(1.0, max(0.0, x))

In [ ]:
def imshow_array(array):
    """Отображение массива нормированных пикселей."""
    plt.axis('off')
    plt.imshow((255.0 * array).astype(np.uint8), cmap=plt.get_cmap("gray"), vmin=0, vmax=255)

In [ ]:
def dataset_Y_to_X(X, Y):
    """Поменять у датасета пары (X, Y) на (X, X) (нужно, например, для обучения автоэнкодера)."""
    return X, X

In [ ]:
def similarity_loss(y_true, y_pred):
    """Функция потерь, которая показала результаты лучше, чем MAE."""
    delta = tf.keras.backend.abs(y_true - y_pred)
    squared = tf.keras.backend.square(y_true - y_pred)
    return tf.keras.backend.mean(delta - 0.5 * squared, axis=-1)

In [ ]:
import json
import csv

info = dict()

## Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = "/content/drive/My Drive/MNIST-Information/"

# Эксперимент для синтетических данных

In [ ]:
import scipy.stats as sps

In [ ]:
def norm_generator(size=1, loc=0, scale=1):
    return sps.norm(loc=loc, scale=scale).rvs(size=size)

In [ ]:
dataset_dim = 16 # Размерность данных.
latent_dim  = 8  # Реальная (скрытая) размерность данных.
final_noize_scale = 0.00 # Стандартное отклонение шума, складываемого с выходом функции.
samples_number = 60000 # Размер выборки.
tests_number   = 10000 # Размер тестовой выборки.

### Функции, задающие многообразия.

In [ ]:
# ЗАГЛУШКА!
def function_1(X, dataset_dim, latent_dim):
    """
    Функция 2, задающая малоразмерное многообразие.
    """

    Y = np.zeros(dataset_dim)

    for i in range(dataset_dim):
        Y[i] = np.sin(np.sin(2 * X[i % latent_dim]) + np.sin(2 * X[i // latent_dim]))

    return Y

$$
y = f_2(x): \quad y_i = \sin \left ( \sin (2 \cdot x_{i \; \text{mod} \; d}) + \sin(2 \cdot x_{[i / d]}) \right )
$$

In [ ]:
def function_2(X, dataset_dim, latent_dim):
    """
    Функция 2, задающая малоразмерное многообразие.
    """

    Y = np.zeros(dataset_dim)

    for i in range(dataset_dim):
        Y[i] = np.sin(np.sin(2 * X[i % latent_dim]) + np.sin(2 * X[i // latent_dim]))

    return Y

$$
y = f_3(x): \quad y_i = \sin \left ( \pi \cdot \tanh \left [ \left ( -3 x_{(i / d^2) \; \text{mod} \; d}^3 + 2 x_{(i / d) \; \text{mod} \; d}^2 + 4 x_{i \; \text{mod} \; d} \right ) / 10 \right ] \right )
$$

In [ ]:
def function_3(X, dataset_dim, latent_dim):
    """
    Функция 3, задающая малоразмерное многообразие.
    """

    Y = np.zeros(dataset_dim)

    for i in range(dataset_dim):
        Y[i] = np.sin(1.0 * np.pi * np.tanh(0.1 * (-3.0 * X[(i // latent_dim**2) % latent_dim]**3 + 2.0 * X[(i // latent_dim) % latent_dim]**2 + 4.0 * X[i % latent_dim])))

    return Y

In [ ]:
functions = [function_1, function_2, function_3]

# НОМЕР ФУНКЦИИ.
# #
# #
function_number = 3 - 1 # -1 для того, чтобы можно было нумеровать функции с единицы и не совершать глупых ошибок...
# #
# #

func_path = path + "Synthetic/function_" + str(function_number + 1) + "/" + str(latent_dim) + "_" + str(dataset_dim) + '/' + ("%.3e" % final_noize_scale) + "/" + str(samples_number) + "_" + str(tests_number) + "/"

In [ ]:
def gen_samples(function, samples_number, dataset_dim, latent_dim, ln_scale = 1.0, fn_scale = 0.05):
    """
    Генерация набора данных.
    """

    # Шум во внутреннем представлении.
    #np.random.seed(42)
    W = norm_generator(loc=0.0, scale=ln_scale, size=(samples_number, latent_dim))
    
    # Отображение шума в пространство большей размерности.
    base_sample = np.zeros((samples_number, dataset_dim))
    for i in range(samples_number):
        base_sample[i] = function(W[i], dataset_dim, latent_dim)
    
    # Дополнительный шум, накладываемый на итоговое многообразие.
    noises_sample = norm_generator(loc=0, scale=fn_scale, size=(samples_number, dataset_dim))
    sample = base_sample + noises_sample
    
    # Обрезание результатов.
    #for i in range(samples_number):
    #    for j in range(dataset_dim):
    #        if sample[i][j] > 1:
    #            sample[i][j] = 1
    #        if sample[i][j] < -1:
    #            sample[i][j] = -1
            
    return sample

In [ ]:
def draw_function():
    samples = gen_samples(functions[function_number], 10000, 16, 4, fn_scale=0.0)

    fig = plt.figure()
    fig.set_figheight(16)
    fig.set_figwidth(30)

    ax = fig.gca(projection='3d')

    X = [samples[i][0] for i in range(len(samples))]
    Y = [samples[i][1] for i in range(len(samples))]
    Z = [samples[i][2] for i in range(len(samples))]

    ax.scatter(X, Y, Z, label='Множество')
    ax.legend()

    plt.show()

    pp = sns.pairplot(pd.DataFrame(samples), height = 2.0, aspect=1.6,
                  plot_kws=dict(edgecolor="k", linewidth=0.0, alpha=0.1, size=0.1),
                  diag_kind="kde", diag_kws=dict(shade=True))

    fig = pp.fig 
    fig.subplots_adjust(top=0.93, wspace=0.3)
    t = fig.suptitle('Wine Attributes Pairwise Plots', fontsize=14)

#draw_function()

In [ ]:
samples = gen_samples(functions[function_number], samples_number, dataset_dim, latent_dim, fn_scale = final_noize_scale)
test = gen_samples(functions[function_number], tests_number, dataset_dim, latent_dim, fn_scale = final_noize_scale)

### Анализ функции

In [ ]:
import os
rel_path = func_path + "No model/"
os.makedirs(rel_path, exist_ok=True)

In [ ]:
pca_codes_dim = dataset_dim
pca_codes = PCA(n_components=pca_codes_dim, whiten=True)
codes_pca = np.array(pca_codes.fit_transform(samples))

In [ ]:
info['singular_values'] = list(pca_codes.singular_values_)
print("Сингулярные значения: \n", pca_codes.singular_values_)

In [ ]:
info['explained_variance'] = list(pca_codes.explained_variance_)
print("Объяснённая вариативность: \n", pca_codes.explained_variance_)

In [ ]:
info['explained_variance_ratio'] = list(pca_codes.explained_variance_ratio_)
print("Относительная объяснённая вариативность: \n", pca_codes.explained_variance_ratio_)

In [ ]:
# Сохранение информации.
with open(rel_path + 'info.json', 'w') as fp:
    json.dump(info, fp)

In [ ]:
lowdim_pca_codes_dim = latent_dim
lowdim_pca_codes = PCA(n_components = lowdim_pca_codes_dim, whiten=True)
lowdim_codes_pca = np.array(lowdim_pca_codes.fit_transform(samples))

In [ ]:
# Средняя абсолютная ошибка по тестовой выборке.

lowdim_pca_codes_test = lowdim_pca_codes.transform(test)
mae = np.mean([np.abs(test[i] - lowdim_pca_codes.inverse_transform(lowdim_pca_codes_test[i])) for i in range(tests_number)])
print(mae)

### KDE для кодов

In [ ]:
# Загрузка параметров KDE.

#with open(rel_path + 'info.json', 'r') as fp:
#    info = json.load(fp)

#kde_codes = KernelDensity(bandwidth=info['bandwidth'], kernel='gaussian')
#kde_codes.fit(codes_pca)

In [ ]:
def smart_gridsearch(begin, end, resolution = 7, rel_x_epsilon = 0.01, rtol = 0.001, n_jobs = 2, cv = 5):
    while True:
        grid = np.logspace(np.log10(begin), np.log10(end), resolution)
        print("Поиск по сетке: ", grid)
        params = {'bandwidth': grid}
        
        grid_search = GridSearchCV(KernelDensity(rtol = rtol), params, n_jobs = n_jobs, verbose = 10, cv = cv)
        grid_search.fit(codes_pca)
        
        if grid_search.best_index_ == 0:
            begin *= begin / end
            end = grid[1]
        elif grid_search.best_index_ == resolution - 1:
            end *= end / begin
            begin = grid[-2]
        else:
            begin = grid[grid_search.best_index_ - 1]
            end = grid[grid_search.best_index_ + 1]

            if end - begin < rel_x_epsilon * grid[grid_search.best_index_]:
                return grid_search 

In [ ]:
kde_codes = smart_gridsearch(0.3, 0.5).best_estimator_
kde_codes.set_params(rtol = 0.0)
print(kde_codes.get_params())

In [ ]:
info['bandwidth'] = kde_codes.get_params()['bandwidth']

# Сохранение информации.
with open(rel_path + 'info.json', 'w') as fp:
    json.dump(info, fp)

## Подсчёт энтропии

In [ ]:
import math

In [ ]:
def entropy_monte_carlo(kde, N, random_state = 42):
    samples  = kde.sample(N, random_state)
    log_prob = np.array(kde.score_samples(samples))
    
    average = -math.fsum(log_prob) / N

    squared_deviations = np.zeros(N)
    for i in range(N):
        squared_deviations[i] = (log_prob[i] - average)**2

    standard_deviation = np.sqrt(math.fsum(squared_deviations) / (N * (N - 1)))

    return average, standard_deviation

In [ ]:
entropy, entropy_error = entropy_monte_carlo(kde_codes, len(codes_pca))
entropy_error *= 3.3 # Коэффициент Стьюдента.
print("H: %f, errH: %f" % (entropy, entropy_error))

In [ ]:
info['MC entropy'] = entropy
info['MC entropy error'] = entropy_error

# Сохранение информации.
with open(rel_path + 'info.json', 'w') as fp:
    json.dump(info, fp)

In [ ]:
import math

def _lvo_step(bandwidth, samples, i):
    lvo_samples = samples
    np.delete(lvo_samples, i)
    
    kde = KernelDensity(bandwidth=bandwidth, kernel='gaussian')
    kde.fit(lvo_samples)
    return kde.score_samples([samples[i]])[0]

In [ ]:
def entropy_leave_one_out_parallel(bandwidth, samples, path, random_state = 42, first_N = None, parts = 10, recover_saved = False):
    # Создание временных папок для сохранения прогресса.
    parts_path = path + "LOO_PARTS/"
    os.makedirs(parts_path, exist_ok=True)

    # Если дано first_N, энтропия будет оцениваться только на первых first_N элементах.
    N = 0
    if first_N is None:
        N = len(samples)
    else:
        N = first_N

    # Число частей и массив, их содержащий.
    N_per_part = N // parts
    log_probs = []

    # Восстанавливаем прогресс, если требуется.
    recovered_parts = 0
    if recover_saved:
        for filename in os.listdir(parts_path):
            if filename.endswith(".csv"):
                log_probs.append(np.loadtxt(parts_path + filename))
                recovered_parts += 1

    print("Восстановлено блоков данных: %d" % recovered_parts)

    # Подсчёт логарифма вероятности в точках.
    for part in range(recovered_parts, parts):
        log_probs.append(np.array(Parallel(n_jobs=2, verbose=10, batch_size=8)(delayed(_lvo_step)(bandwidth, samples, i) for i in range(part * N_per_part, min((part + 1) * N_per_part, N)))))
        np.savetxt(parts_path + str(part) + ".csv", log_probs[part], delimiter="\n")
    
    # Объединение в один массив.
    log_prob = np.concatenate(log_probs)

    # Суммирование и нахождение стандартного отклонения.
    average = -math.fsum(log_prob) / N    
    squared_deviations = np.zeros(N)
    for i in range(N):
        squared_deviations[i] = (log_prob[i] - average)**2
    standard_deviation = np.sqrt(math.fsum(squared_deviations) / (N * (N - 1)))
        
    return average, standard_deviation

In [ ]:
entropy, entropy_error = entropy_leave_one_out_parallel(kde_codes.get_params()['bandwidth'], codes_pca, rel_path, recover_saved = False)
entropy_error *= 3.3 # Коэффициент Стьюдента.
print("H: %f, errH: %f" % (entropy, entropy_error))

In [ ]:
info['LOO entropy'] = entropy
info['LOO entropy error'] = entropy_error

# Сохранение информации.
with open(rel_path + 'info.json', 'w') as fp:
    json.dump(info, fp)

## Оценка размерности кодов

In [ ]:
tree_codes = BallTree(codes_pca, leaf_size=40)

In [ ]:
def calc_pairs(tree, samples, radius):
    total = sum(tree.query_radius(samples, r=radius, count_only=True)) - len(samples)
    return total // 2

In [ ]:
max_pairs = len(codes_pca) * (len(codes_pca) - 1) // 2
print(max_pairs)

In [ ]:
def ineq_binary_search(func, a, b, rel_eps = 0.01, max_a = None, min_b = None, verbose = 0):
    while True:
        print("Уточнение параметров: [%.3e, %.3e]" % (a, b))
        if (a != 0.0) and (not func(a)):
            a /= 2.0
        elif func(b):
            b *= 2.0
        else:
            break

    while np.abs(1 - a / b) > rel_eps:
        if verbose > 0:
            print("Бинарный поиск: [%.3e, %.3e]" % (a, b))
        
        pos = (a + b) / 2

        if func(pos):
            if (not (max_a is None)) and max_a < pos:
                a = max_a
                b = pos
                break
            a = pos

        else:
            if (not (min_b is None)) and min_b > pos:
                a = pos
                b = min_b
                break
            b = pos

    return a, b

In [ ]:
# Начальное предположение.
min_radius_a = 0.0
min_radius_b = 0.001
max_radius_a = 15.00
max_radius_b = 25.00

min_radius_a, min_radius_b = ineq_binary_search(lambda x: calc_pairs(tree_codes, codes_pca, x) < 1, min_radius_a, min_radius_b, min_b = 1.0e-03, verbose = 1)
max_radius_a, max_radius_b = ineq_binary_search(lambda x: calc_pairs(tree_codes, codes_pca, x) < max_pairs, max_radius_a, max_radius_b, verbose = 1)

min_radius = min_radius_a
max_radius = max_radius_b

#assert calc_pairs(tree_codes, codes_pca, min_radius) == 0
#assert calc_pairs(tree_codes, codes_pca, max_radius) == max_pairs

print("min_radius: %.3e; pairs: %d" % (min_radius, calc_pairs(tree_codes, codes_pca, min_radius_b)))
print("max_radius: %.3e; pairs: %d" % (max_radius, calc_pairs(tree_codes, codes_pca, max_radius_a)))

In [ ]:
info['min_radius'] = min_radius
info['max_radius'] = max_radius

# Сохранение информации.
with open(rel_path + 'info.json', 'w') as fp:
    json.dump(info, fp)

In [ ]:
resolution = 128

grid = np.logspace(np.log10(min_radius), np.log10(max_radius), resolution)
pairs = np.zeros(resolution, dtype='int64')
for i in range(resolution):
    pairs[i] = calc_pairs(tree_codes, codes_pca, grid[i])
    print("(%d/%d): %f, %d" % (i+1, resolution, grid[i], pairs[i]))
    
    #if pairs[i] == max_pairs:
    #    break

In [ ]:
writer = csv.writer(open(rel_path + "pairs.csv", 'w'))
for i in range(resolution):
    writer.writerow([grid[i], pairs[i]])

In [ ]:
log_grid__pairs = np.column_stack((grid, pairs))
for i in range(resolution):
    log_grid__pairs[i][0] = np.log(log_grid__pairs[i][0])
    log_grid__pairs[i][1] = np.log(log_grid__pairs[i][1] / max_pairs)

np.savetxt(rel_path + "log_pairs.csv", log_grid__pairs, delimiter=",", newline='\n')